In this notebook, we will build a 3D map of a scene from a small set of images and then localize an image downloaded from the Internet. This demo was contributed by [Philipp Lindenberger](https://github.com/Phil26AT/).

In [ ]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook
import os, sys
BASEPATH = os.getcwd().split('code')[-2]
sys.path.append(os.path.join(BASEPATH, 'code'))
from utils import pairs_from_sequence, pairs_from_360

tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from pathlib import Path
import numpy as np
import pycolmap
import copy

from hloc import (
    extract_features,
    match_features,
    reconstruction,
    visualization,
    pairs_from_exhaustive
)
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d
import pycolmap

from utils.multiview_utils import Camera, Calibration
from utils.metadata_utils import get_cam_names

# Setup
Here we define some output paths.

In [ ]:
arv_copy = sys.argv
sys.argv = ['pop']
sys.argv.append('-cfg')
config_path = os.path.abspath('../../code/configs/project_config.yaml')
sys.argv.append(config_path)
sys.argv.append('-dr')
root_path = os.path.abspath('../../data/')
sys.argv.append(root_path)
sys.argv.append('-l')
sys.argv.append('info')

RESTART = False # set true if wanting to run everything from scratch


images = Path("../../data/0-calibration/images")
outputs = Path("../../data/0-calibration/outputs/")
if RESTART:
    if os.path.exists(output_directory):
        # Remove all contents and the directory itself
        shutil.rmtree(output_directory)
        print(f"Removed directory and its contents: {output_directory}")
    else:
        print(f"Directory does not exist: {output_directory}")
# outputs.mkdir(exist_ok=True)
sfm_pairs = outputs / "pairs-sfm.txt"
loc_pairs = outputs / "pairs-loc.txt"
sfm_dir = outputs / "sfm"
features = outputs / "features.h5"
matches = outputs / "matches.h5"

feature_conf = extract_features.confs["disk"]
matcher_conf = match_features.confs["disk+lightglue"]
# feature_conf = extract_features.confs["superpoint_aachen"]
# matcher_conf = match_features.confs["superpoint+lightglue"]

# 3D mapping
First we list the images used for mapping. These are all day-time shots of Sacre Coeur.

In [ ]:
# references = [p.relative_to(images).as_posix() for p in (images).iterdir() if '360' in p.name]
references = [p.relative_to(images).as_posix() for p in images.rglob('*.jpg') if '360' in p.parent.name]
print(len(references), "mapping images")
# plot_images([read_image(images / r) for r in references], dpi=25)

Then we extract features and match them across image pairs. Since we deal with few images, we simply match all pairs exhaustively. For larger scenes, we would use image retrieval, as demonstrated in the other notebooks.

In [ ]:
if RESTART:

    extract_features.main(
        feature_conf, images, image_list=references, feature_path=features
    )

    pairs_from_360.main(sfm_pairs, image_dir=images, window_size=4)

    match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches);

The we run incremental Structure-From-Motion and display the reconstructed 3D model.

In [ ]:
# opts = dict(camera_model = "SIMPLE_RADIAL", camera_params =','.join(map(str, (1, 256, 256, 0))))
if RESTART:
    model = reconstruction.main(
        sfm_dir, images, sfm_pairs, features, matches, image_list=references, camera_mode=pycolmap.CameraMode.PER_FOLDER)

In [ ]:
model_dir = outputs / 'full_reconstruction/'

model_dir.mkdir(exist_ok = True, parents = True)
if RESTART:
    model.write(model_dir)

# load reconstruction

model = pycolmap.Reconstruction(model_dir)

# Localise Static Cams

In [ ]:
query = [p.relative_to(images).as_posix() for p in images.rglob('*.jpg') if '360' not in p.parent.name]

extract_features.main(
    feature_conf, images, image_list=query, feature_path=features, overwrite=True
)
pairs_from_exhaustive.main(loc_pairs, image_list=query, ref_list=references)


match_features.main(
    matcher_conf, loc_pairs, features=features, matches=matches, overwrite=True
);

In [ ]:
import pycolmap
from hloc.localize_sfm import QueryLocalizer, pose_from_cluster
from utils.colmap_utils import add_camera_and_image, camera_calibs_from_colmap


# camera = pycolmap.infer_camera_from_image(images / query)
ref_ids = [model.find_image_with_name(r).image_id for r in references if model.find_image_with_name(r) is not None]
conf = {
    "estimation": {"ransac": {"max_error": 12}},
    "refinement": {"refine_focal_length": False, "refine_extra_params": False},
}
localizer = QueryLocalizer(model, conf)
rets = []
logs = []
for query_ in query:
    camera = pycolmap.infer_camera_from_image(images / query_)
    camera.params = [640, 640, 340, 0]
    ret, log = pose_from_cluster(localizer, query_, camera, ref_ids, features, matches)
    ret['image'] = query_
    # add results to reconstruction
    try:
        add_camera_and_image(model, ret['camera'], ret["cam_from_world"], ret["image"])
    except ValueError as e:
        print(e)

    rets.append(ret)
    logs.append(log)
    # visualise found matches for verification
    print(f'found {ret["num_inliers"]}/{len(ret["inliers"])} inlier correspondences.')
    visualization.visualize_loc_from_log(images, query_, log, model)




In [ ]:



# Define necessary functions for ray-plane intersection
def ray_plane_intersection(ray_origin, ray_direction, plane_normal, plane_point):
    d = np.dot(plane_normal, plane_point)
    t = (d - np.dot(plane_normal, ray_origin)) / np.dot(plane_normal, ray_direction)
    return ray_origin + t * ray_direction

def image_to_world_coordinates(cam, roi_points, z_plane=0):
    intrinsic = cam.calibration.intrinsic  # Expecting a 3x3 numpy array
    rotation = cam.rotation_matrix()  # Rotation matrix of the camera
    translation = cam.translation_vector()  # Translation vector of the camera
    
    plane_normal = np.array([0, 0, 1])
    plane_point = np.array([0, 0, z_plane])

    world_points = []
    for point in roi_points:
        x, y = point
        # Convert image coordinates to normalized camera coordinates
        normalized_coords = np.linalg.inv(intrinsic) @ np.array([x, y, 1])
        # Convert normalized camera coordinates to world coordinates
        ray_direction = rotation @ normalized_coords
        ray_origin = translation
        # Find the intersection of the ray with the plane z=0
        world_point = ray_plane_intersection(ray_origin, ray_direction, plane_normal, plane_point)
        world_points.append(world_point)
    
    return np.array(world_points)


def plot_reconstruction(model):

    """plots a pycolmap reconstruction and adds regions of interest for the cameras
    """
    config = get_config_dict()
    fig = viz_3d.init_figure()
    
    viz_3d.plot_reconstruction(
    fig, temp_model, color="rgba(255,0,0,0.5)", name="mapping", points_rgb=True
    )

    mvvids = MultiviewVids(newest=False, config=config)
    mvvids.cams = camera_calibs_from_colmap(images, temp_model, save=False)

    bbx = model.compute_bounding_box(0.01, 0.99)

    # Add a static plane at z=0
    plane = go.Mesh3d(
        x=[bbx[0][0], bbx[1][0], bbx[1][0], bbx[0][0]],
        y=[bbx[0][1], bbx[0][1], bbx[1][1], bbx[1][1]],
        z=[0, 0, 0, 0],
        color='lightblue',
        opacity=0.5,
        name='Plane at z=0',
        showscale=False
    )

    fig.add_trace(plane)

    # Process the cameras and plot the ROI intersection with the plane z=0
    for cam in mvvids.cams:
        roi_image_coords = cam.calibration.ROI
        roi_camera_coords = cam.from_image(roi_image_coords)
        roi_world_coords = cam.convert_to_world_frame(roi_camera_coords)

        # Get the camera center in world coordinates
        camera_center = cam.get_position()

        plane_normal = np.array([0, 0, 1])
        plane_point = np.array([0, 0, 0])

        for point in roi_world_coords:
            ray_direction = point - camera_center
            intersection_point = ray_plane_intersection(camera_center, ray_direction, plane_normal, plane_point)
            
            # Plot the extended ray
            line = go.Scatter3d(
                x=[camera_center[0], intersection_point[0]],
                y=[camera_center[1], intersection_point[1]],
                z=[camera_center[2], intersection_point[2]],
                mode='lines',
                line=dict(color='blue', width=2),
                name=f'Ray through ROI point {cam.name}'
            )
            fig.add_trace(line)

    
    
    return fig



def plot_ground_plane_proj(model, exclude:list = []):
    """
    Plots the groundplane projection for the set of static cameras in the reconstruction
    """
    # Initialize configuration and multiview video handler
    config = get_config_dict()
    mvvids = MultiviewVids(newest=False, config=config)
    cameras = camera_calibs_from_colmap(images, temp_model, save=False)
    mvvids.cams = [camera for camera in cameras if camera.name not in exclude]

    # Extract frames
    max_frame = np.min([10, mvvids.get_max_frame_id() - 1])
    step = 2
    frame_ids = list(np.arange(0, max_frame, step))
    base_frames = mvvids.extract_mv(frame_ids, undistort=True)

    # Setup plotly figure for 2D image projection
    fig_img = go.Figure()

    alphas = [1.0, 0.7, 0.5, 0.3]
    rect, _ = mvvids.get_bounding_box()
    output_img_size = (1920, 1080)
    input_img_size = (1080, 1920)

    for cam in mvvids.cams:
        img = copy.deepcopy(base_frames[cam.name][0])
        img = cv2.resize(img, (input_img_size[1], input_img_size[0]))
        H = cam.get_ground_plane_homography(input_img_size=input_img_size, output_img_size=output_img_size, bounding_box=rect, padding_percent=0)
        new_img = project_to_ground_plane_cv2(img, H, output_img_size)

        fig_img.add_trace(go.Image(z=new_img, opacity=alphas[int(cam.calibration.view_id) - 1]))


    return fig_img

def plot_histogram(model):
    """
    Plots the distribution of z-coordinates for a given pycolmap reconstruction
    """
    points = np.array([point.xyz for point in model.points3D.values()])
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=points[:, 2], name='Z-Value Distribution'))

    return fig
    

In [ ]:
from utils.multiview_utils import MultiviewVids
from utils.metadata_utils import get_config_dict
from hloc.utils.viz_3d import plot_camera
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import copy
import numpy as np
import pycolmap
from IPython.display import display
from utils.coordinate_utils import project_to_ground_plane_cv2
import ipywidgets as widgets
from IPython.display import display
import cv2


temp_model = copy.deepcopy(model)
# Transform the model
temp_model.transform(pycolmap.Sim3d(1, pycolmap.Rotation3d([-0.7071068, 0, 0, 0.7071068]), [0, 0, 0]))
temp_model.transform(pycolmap.Sim3d(1, pycolmap.Rotation3d([0, 0, 0, 1]), [0, 0, -temp_model.compute_bounding_box(0.01, 0.99)[0][2]]))


# Create the initial subplot layout
fig_subplots = make_subplots(
    rows=3, cols=1, 
    subplot_titles=("Reconstruction", "Ground Plane", "Z-Histogram"),
    specs=[[{'type': 'scene'}], [{'type': 'xy'}], [{'type': 'xy'}]]
)


def rerender_plots(new_z):
    new_recon = copy.deepcopy(temp_model)
    new_recon.transform(pycolmap.Sim3d(1, pycolmap.Rotation3d([0, 0, 0, 1]), [0, 0, new_z]))
    fig_reconstruction = plot_reconstruction(new_recon)

    fig_subplots.data = []

    # Add extracted 3D traces to the first subplot (row=1, col=1)
    for trace in fig_reconstruction.data:
        fig_subplots.add_trace(trace, row=1, col=1)

    fig_img = plot_ground_plane_proj(new_recon, exclude=['cam1'])
    for trace in fig_img.data:
        fig_subplots.add_trace(trace, row=2, col=1)

    fig_hist = plot_histogram(new_recon)
    for trace in fig_hist.data:
        fig_subplots.add_trace(trace, row=3, col=1)

rerender_plots(0)
# Display the figure
recon_widget = go.FigureWidget(fig_subplots)
display(recon_widget)

# Define a button to trigger the update
button = widgets.Button(description="Rerender")

# Define the button click event
def on_button_click(b):
    rerender_plots(z_global)

# Bind the button click event
button.on_click(on_button_click)


# Define the initial value for z
z_global = 0

# Create a slider widget for z
z_slider = widgets.FloatSlider(
    value=z_global,
    min=-10,
    max=10,
    step=0.1,
    description='Z Value:',
    continuous_update=False
)

# Define a function to update the plane position based on the slider value
def update_z(change):
    z_global = change['new']
    with recon_widget.batch_update():
                for trace in recon_widget.data:
                    if trace.name == 'Plane at z=0':
                        trace.z += z_global

# Attach the update function to the slider
z_slider.observe(update_z, names='value')
display(z_slider)
display(button)


In [ ]:
# NOTE: For Downsampling
# from hloc.utils.viz_3d import plot_points, plot_cameras
# import random
# def plot_reconstruction_tight(
#     fig: go.Figure,
#     rec: pycolmap.Reconstruction,
#     max_reproj_error: float = 6.0,
#     color: str = "rgb(0, 0, 255)",
#     name = None,
#     min_track_length: int = 2,
#     points: bool = True,
#     cameras: bool = True,
#     points_rgb: bool = True,
#     cs: float = 1.0,
#     ratio = .1
# ):
#     # Filter outliers
#     bbs = rec.compute_bounding_box(0.001, 0.999)
#     # Filter points, use original reproj error here
#     p3Ds = [
#         p3D
#         for _, p3D in rec.points3D.items()
#         if (
#             (p3D.xyz >= bbs[0]).all()
#             and (p3D.xyz <= bbs[1]).all()
#             and p3D.error <= max_reproj_error
#             and p3D.track.length() >= min_track_length
#         )
#     ]
#     print(len(p3Ds))
#     p3Ds = random.sample(p3Ds, int(ratio * len(p3Ds)))
#     xyzs = [p3D.xyz for p3D in p3Ds]
#     if points_rgb:
#         pcolor = [p3D.color for p3D in p3Ds]
#     else:
#         pcolor = color
#     if points:
#         plot_points(fig, np.array(xyzs), color=pcolor, ps=1, name=name)
#     if cameras:
#         plot_cameras(fig, rec, color=color, legendgroup=name, size=cs)

In [ ]:
# NOTE: For Rotation of groundplane
# # Function to update the plot based on Euler angles and translation
# def update_plot(roll, pitch, yaw, trans_x, trans_y, trans_z):
#     # Convert Euler angles to quaternion
#     r = R.from_euler('xyz', [roll, pitch, yaw], degrees=True)
#     quat = r.as_quat()  # Returns in the order [x, y, z, w]
    
#     # Create the transformation object
#     rig3d = pycolmap.Rigid3d()
#     rig3d.rotation = pycolmap.Rotation3d(quat)
#     rig3d.translation = [trans_x, trans_y, trans_z]
    
#     # Get the transformation matrix (4x3)
#     transform_matrix = rig3d.matrix()
    
#     # Extract rotation and translation
#     rotation_matrix = transform_matrix[:, :3]
#     translation_vector = transform_matrix[:, 3]
    
#     # Define the axes in the original frame
#     original_axes = {
#         'x': np.array([[0, 0, 0], [1, 0, 0]]),
#         'y': np.array([[0, 0, 0], [0, 1, 0]]),
#         'z': np.array([[0, 0, 0], [0, 0, 1]])
#     }
    
#     # Apply the transformation to the axes
#     transformed_axes = {
#         axis: np.dot(points, rotation_matrix.T) + translation_vector
#         for axis, points in original_axes.items()
#     }
    
#     # Update the traces in the figure
#     with fig.batch_update():
#         fig.data[0].x, fig.data[0].y, fig.data[0].z = transformed_axes['x'][:, 0], transformed_axes['x'][:, 1], transformed_axes['x'][:, 2]
#         fig.data[1].x, fig.data[1].y, fig.data[1].z = transformed_axes['y'][:, 0], transformed_axes['y'][:, 1], transformed_axes['y'][:, 2]
#         fig.data[2].x, fig.data[2].y, fig.data[2].z = transformed_axes['z'][:, 0], transformed_axes['z'][:, 1], transformed_axes['z'][:, 2]
#     fig.show()

# # Create sliders for the Euler angles and translation components
# roll_slider = widgets.FloatSlider(value=0, min=-180, max=180, step=1, description='Roll')
# pitch_slider = widgets.FloatSlider(value=0, min=-180, max=180, step=1, description='Pitch')
# yaw_slider = widgets.FloatSlider(value=0, min=-180, max=180, step=1, description='Yaw')
# trans_x_slider = widgets.FloatSlider(value=0, min=-5, max=5, step=0.1, description='Trans X')
# trans_y_slider = widgets.FloatSlider(value=0, min=-5, max=5, step=0.1, description='Trans Y')
# trans_z_slider = widgets.FloatSlider(value=0, min=-5, max=5, step=0.1, description='Trans Z')

# # Interactive display
# def update(*args):
#     update_plot(roll_slider.value, pitch_slider.value, yaw_slider.value, 
#                 trans_x_slider.value, trans_y_slider.value, trans_z_slider.value)

# roll_slider.observe(update, names='value')
# pitch_slider.observe(update, names='value')
# yaw_slider.observe(update, names='value')
# trans_x_slider.observe(update, names='value')
# trans_y_slider.observe(update, names='value')
# trans_z_slider.observe(update, names='value')

# # Display widgets
# widgets.VBox([roll_slider, pitch_slider, yaw_slider, trans_x_slider, trans_y_slider, trans_z_slider])